In [6]:
import pandas as pd
from pandas import json_normalize

In [7]:
df = pd.read_json("people_more_data.json")
df.head()

,name,age,address,contacts,languages,interests
0,Alice,30,"{'street': '123 Main St', 'city': 'New York', ...","[{'type': 'email', 'value': 'alice@example.com...","[English, Spanish]","{'hobbies': ['Reading', 'Hiking'], 'sports': [..."
1,Bob,25,"{'street': '456 Elm St', 'city': 'Los Angeles'...","[{'type': 'email', 'value': 'bob@example.com'}...","[English, French]","{'hobbies': ['Cooking', 'Gardening'], 'sports'..."
2,Charlie,35,"{'street': '789 Oak St', 'city': 'Chicago', 'z...","[{'type': 'email', 'value': 'charlie@example.c...","[English, German]","{'hobbies': ['Painting', 'Photography'], 'spor..."
3,David,28,"{'street': '567 Pine St', 'city': 'San Francis...","[{'type': 'email', 'value': 'david@example.com...","[English, Spanish]","{'hobbies': ['Traveling', 'Cooking'], 'sports'..."
4,Eva,32,"{'street': '890 Oakwood Dr', 'city': 'Miami', ...","[{'type': 'email', 'value': 'eva@example.com'}...","[English, French]","{'hobbies': ['Reading', 'Painting'], 'sports':..."


In [8]:
df_address = df['address']
df_contacts = df['contacts']
df_languages = df['languages']
df_interests = df['interests']

df_address.head()

0    {'street': '123 Main St', 'city': 'New York', ...
1    {'street': '456 Elm St', 'city': 'Los Angeles'...
2    {'street': '789 Oak St', 'city': 'Chicago', 'z...
3    {'street': '567 Pine St', 'city': 'San Francis...
4    {'street': '890 Oakwood Dr', 'city': 'Miami', ...
Name: address, dtype: object

In [9]:
df = df.drop(['address','contacts','languages','interests'], axis=1)

In [10]:
df_flatt = df.copy()
list_of_dataframes = [df_address, df_contacts, df_languages, df_interests]

for dataframe in list_of_dataframes:
    # Check if the series contains dictionaries.
    if dataframe.apply(lambda x: isinstance(x, dict)).all():
        # Flatten dictionaries using json_normalize
        df_temp = json_normalize(dataframe)
        df_temp.columns = [str(col) for col in df_temp.columns]
        df_flatt = pd.concat([df_flatt, df_temp], axis=1)
    # Check if the series contains lists.
    elif dataframe.apply(lambda x: isinstance(x, list)).all():
        first_element = dataframe.iloc[0]
        if all(isinstance(item, str) for item in first_element):  # Assuming consistent data
            # Convert lists of strings to dummy columns
            df_temp = dataframe.str.join('|').str.get_dummies()
            df_temp.columns = [str(col) for col in df_temp.columns]
            df_flatt = pd.concat([df_flatt, df_temp], axis=1)
        else:
            # For lists of dictionaries like 'contacts', you'll need to design a more specialized flattening approach.
            # For example, extracting emails from contacts:
            df_flatt['email'] = dataframe.apply(lambda x: [contact['value'] for contact in x if contact['type'] == 'email'][0])

# Drop the original columns if you desire
df_flatt = df_flatt.drop(['address', 'contacts', 'languages', 'interests'], axis=1, errors='ignore')


In [11]:
#Fix the last part with hobbies and sports still in a list format
hobbies_dummies = df_flatt['hobbies'].str.join('|').str.get_dummies()
hobbies_dummies.columns = ['hobby_' + col for col in hobbies_dummies.columns]

sports_dummies = df_flatt['sports'].str.join('|').str.get_dummies()
sports_dummies.columns = ['sport_' + col for col in sports_dummies.columns]

df_flatt = pd.concat([df_flatt, hobbies_dummies, sports_dummies], axis=1)
df_flatt.drop(['hobbies', 'sports'], axis=1, inplace=True)
df_flatt.head()


,name,age,street,city,zipcode,email,English,French,German,Spanish,...,hobby_Photography,hobby_Reading,hobby_Traveling,hobby_Yoga,sport_Basketball,sport_Cycling,sport_Running,sport_Soccer,sport_Swimming,sport_Tennis
0,Alice,30,123 Main St,New York,10001,alice@example.com,1,0,0,1,...,0,1,0,0,0,0,0,0,1,1
1,Bob,25,456 Elm St,Los Angeles,90001,bob@example.com,1,1,0,0,...,0,0,0,0,1,1,0,0,0,0
2,Charlie,35,789 Oak St,Chicago,60601,charlie@example.com,1,0,1,0,...,1,0,0,0,0,0,1,1,0,0
3,David,28,567 Pine St,San Francisco,94101,david@example.com,1,0,0,1,...,0,0,1,0,0,0,0,1,1,0
4,Eva,32,890 Oakwood Dr,Miami,33101,eva@example.com,1,1,0,0,...,0,1,0,0,0,1,0,0,0,1
